# Problema 4: Geração de Texto com RNN, LSTM e GRU

**Objetivo:** Comparar o desempenho de três arquiteturas de redes neurais recorrentes (`SimpleRNN`, `LSTM` e `GRU`) em uma tarefa de geração de texto.

**Abordagem:**
1.  Carregar e pré-processar o texto de três livros da série Harry Potter.
2.  Utilizar um **Tokenizador** para criar um vocabulário de palavras.
3.  Treinar cada modelo para prever a próxima palavra em uma sequência.
4.  Gerar texto com cada modelo para comparar visualmente a coerência e qualidade.

### 1. Importação das Bibliotecas

Vamos importar todas as bibliotecas necessárias para o projeto.

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, GRU, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os
import gc

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.list_physical_devices('CPU')))
print("TensorFlow version:", tf.__version__)

mixed_precision.set_global_policy('mixed_float16')

Num GPUs Available:  1
Num CPUs Available:  1
TensorFlow version: 2.20.0


### 2. Carregamento e Pré-processamento dos Dados

Carregamos os textos, juntamos em um único corpus e usamos o `Tokenizer` para criar um vocabulário de palavras.

In [23]:
# Carregar e concatenar os textos
text = ""
for i in range(1, 4):
    filepath = os.path.join('dataset', f'harry_potter_{i}.txt')
    with open(filepath, 'r', encoding='utf-8') as f:
        text += f.read()

# --- Tokenização por Palavra ---
tokenizer = Tokenizer(oov_token="<unk>")
tokenizer.fit_on_texts([text])

word_to_int = tokenizer.word_index
int_to_word = {i: w for w, i in word_to_int.items()}
vocab_size = len(word_to_int) + 1
print(f"Tamanho do vocabulário: {vocab_size} palavras")

# Converter todo o texto para uma sequência de inteiros
full_sequence = tokenizer.texts_to_sequences([text])[0]

Tamanho do vocabulário: 16633 palavras


### 3. Criação das Sequências de Treino

Transformamos a longa sequência de palavras em pares de `(entrada, saída)` para o treinamento.

In [24]:
seq_length = 50  # Usaremos 50 palavras para prever a 51ª
X_data = []
y_data = []

for i in range(seq_length, len(full_sequence)):
    in_seq = full_sequence[i-seq_length:i]
    out_word = full_sequence[i]
    X_data.append(in_seq)
    y_data.append(out_word)

n_patterns = len(X_data)
print(f"Total de sequências de treino: {n_patterns}")

# Preparar os dados para a rede neural
X = np.array(X_data)
y = np.array(y_data)

Total de sequências de treino: 291509


### 4. Definição do Modelo e Funções de Apoio

Criamos funções para construir os modelos e gerar texto. Também configuramos os `callbacks` para um treinamento eficiente.

In [26]:
def create_model(recurrent_layer, vocab_size, seq_length):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=100, input_length=seq_length),
        recurrent_layer(128, return_sequences=True),
        recurrent_layer(128),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def generate_text(model, tokenizer, seed_text, num_words_to_gen=40, temperature=0.7):
    full_generated_text = seed_text.lower()
    print(f"Semente: \"{seed_text}\" | Temperatura: {temperature}")
    print("Texto gerado:")
    print("------------------")
    print(seed_text, end=' ')
    
    for _ in range(num_words_to_gen):
        token_list = tokenizer.texts_to_sequences([full_generated_text])[0]
        token_list = pad_sequences([token_list], maxlen=seq_length, padding='pre')
        
        # Previsão e aplicação da temperatura
        prediction = model.predict(token_list, verbose=0)[0]
        prediction = np.log(prediction + 1e-7) / temperature # Adicionado 1e-7 para evitar log(0)
        
        # Amostragem da próxima palavra
        index = tf.random.categorical(prediction[np.newaxis, :], num_samples=1)[0, 0].numpy()
        
        output_word = int_to_word.get(index, "<unk>")
        full_generated_text += " " + output_word
        print(output_word, end=' ')
    print("\n------------------\n")

# Parâmetros de treino
epochs = 100
seed = "harry potter olhou para o castelo e"

### 5. Treinamento e Avaliação dos Modelos

Treinamos cada modelo sequencialmente, gerando o texto e limpando a memória da GPU após cada um para evitar sobrecarga.

In [ ]:
# --- Modelo 1: SimpleRNN ---
print("### Treinando o Modelo SimpleRNN ###")
rnn_model = create_model(SimpleRNN, vocab_size, seq_length)
rnn_model.fit(X, y, epochs=epochs, batch_size=128, verbose=1, validation_split=0.1)

print("\n--- Geração com SimpleRNN ---")
generate_text(rnn_model, tokenizer, seed)

# Salvar o modelo
rnn_model.save('modelo_simplernn_harry_potter.keras')
print("✓ Modelo SimpleRNN salvo como 'modelo_simplernn_harry_potter.keras'")

del rnn_model
tf.keras.backend.clear_session()
gc.collect()

In [ ]:
# --- Modelo 2: LSTM ---
print("\n### Treinando o Modelo LSTM ###")
lstm_model = create_model(LSTM, vocab_size, seq_length)
lstm_model.fit(X, y, epochs=epochs, batch_size=128, verbose=1, validation_split=0.1)

print("\n--- Geração com LSTM ---")
generate_text(lstm_model, tokenizer, seed)

# Salvar o modelo
lstm_model.save('modelo_lstm_harry_potter.keras')
print("✓ Modelo LSTM salvo como 'modelo_lstm_harry_potter.keras'")

del lstm_model
tf.keras.backend.clear_session()
gc.collect()


### Treinando o Modelo LSTM ###
Epoch 1/100
2050/2050 ━━━━━━━━━━━━━━━━━━━━ 17s 8ms/step - accuracy: 0.0527 - loss: 6.7612 - val_accuracy: 0.0695 - val_loss: 6.3984 - learning_rate: 0.0010
Epoch 2/100
2050/2050 ━━━━━━━━━━━━━━━━━━━━ 17s 8ms/step - accuracy: 0.0527 - loss: 6.7612 - val_accuracy: 0.0695 - val_loss: 6.3984 - learning_rate: 0.0010
Epoch 2/100
2045/2050 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0742 - loss: 6.2452
Epoch 2: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
2050/2050 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.0839 - loss: 6.1609 - val_accuracy: 0.1024 - val_loss: 6.0772 - learning_rate: 0.0010
Epoch 3/100

Epoch 2: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
2050/2050 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.0839 - loss: 6.1609 - val_accuracy: 0.1024 - val_loss: 6.0772 - learning_rate: 0.0010
Epoch 3/100
2050/2050 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.1071 - loss: 5.8054 - val_accuracy: 0.1111 

0

In [ ]:
# --- Modelo 3: GRU ---
print("\n### Treinando o Modelo GRU ###")
gru_model = create_model(GRU, vocab_size, seq_length)
gru_model.fit(X, y, epochs=epochs, batch_size=128, verbose=1, validation_split=0.1)

print("\n--- Geração com GRU ---")
generate_text(gru_model, tokenizer, seed)

# Salvar o modelo
gru_model.save('modelo_gru_harry_potter.keras')
print("✓ Modelo GRU salvo como 'modelo_gru_harry_potter.keras'")

del gru_model
tf.keras.backend.clear_session()
gc.collect()


### Treinando o Modelo GRU ###
Epoch 1/100
2050/2050 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - accuracy: 0.0534 - loss: 6.8166 - val_accuracy: 0.0819 - val_loss: 6.2980
Epoch 2/100
2050/2050 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - accuracy: 0.1048 - loss: 5.9029 - val_accuracy: 0.1242 - val_loss: 5.8036
Epoch 3/100
2050/2050 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - accuracy: 0.1352 - loss: 5.3408 - val_accuracy: 0.1381 - val_loss: 5.6810
Epoch 4/100
2050/2050 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - accuracy: 0.1534 - loss: 4.9866 - val_accuracy: 0.1421 - val_loss: 5.6758
Epoch 5/100
2050/2050 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - accuracy: 0.1691 - loss: 4.7191 - val_accuracy: 0.1444 - val_loss: 5.7017
Epoch 6/100
2050/2050 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - accuracy: 0.1839 - loss: 4.4964 - val_accuracy: 0.1462 - val_loss: 5.7414
Epoch 7/100
2050/2050 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - accuracy: 0.1991 - loss: 4.3054 - val_accuracy: 0.1459 - val_loss: 5.7920
Epoch 8/100
2050/2050 ━━━━━━━━━━━━━━━━━━━━ 1

0

### 6. Carregar Modelos Salvos e Gerar Novos Textos

Use esta seção para carregar qualquer um dos modelos treinados e gerar texto com diferentes sementes (frases iniciais).

In [ ]:
# Escolha qual modelo carregar (descomente a linha desejada)
modelo_escolhido = 'modelo_lstm_harry_potter.keras'  # Altere para o modelo desejado
# modelo_escolhido = 'modelo_simplernn_harry_potter.keras'
# modelo_escolhido = 'modelo_gru_harry_potter.keras'

# Carregar o modelo
print(f"Carregando o modelo: {modelo_escolhido}")
modelo_carregado = tf.keras.models.load_model(modelo_escolhido)
print("✓ Modelo carregado com sucesso!\n")

# Defina sua própria frase inicial aqui
nova_semente = "hermione abriu o livro e"

# Gerar texto com o modelo carregado
# Você pode ajustar a temperatura para controlar a criatividade:
# - temperature=0.5 : Mais conservador e previsível
# - temperature=0.7 : Balanceado (padrão)
# - temperature=1.0 : Mais criativo e arriscado
generate_text(modelo_carregado, tokenizer, nova_semente, num_words_to_gen=50, temperature=0.7)